# 这个文件是下载并导入的文件

In [1]:
import GEOparse
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
GPL96 = GEOparse.get_GEO(geo='GPL96', destdir="./datasets", silent=True)
GPL570 = GEOparse.get_GEO(geo='GPL570', destdir="./datasets", silent=True)
GPL571 = GEOparse.get_GEO(geo='GPL571', destdir="./datasets", silent=True)
GPL2507 = GEOparse.get_GEO(geo='GPL2507', destdir="./datasets", silent=True)

In [3]:
GSE_IDs = [ "6269",
            "20346",
            "40012",
            "40396",
            "42026",
            "60244",
            "66099",
            "63990",
            "21802",
            "27131",
            "28750",
            "42834",
            "57065",
            "68310",
            "69528",
            "111368"]

In [4]:
for gseid in GSE_IDs:
    try:
        GEONAME="GSE" + gseid
        print(GEONAME)
        gse = GEOparse.get_GEO(geo=GEONAME, destdir="./datasets")
        sample_value = gse.pivot_samples('VALUE').T

        infection = []
        gsm_list = []
        n = 0
        no_new_gpl = 1

        value = pd.DataFrame()
        for i in gse.gsms:
            n += 1

            if n == 1:
                print(gse.gsms[i].metadata['characteristics_ch1'])
            # 感染状态的处理
            characteristics_ch1 = gse.gsms[i].metadata['characteristics_ch1']   # the infection_status is in the characteristics_ch1
            infection_status = characteristics_ch1[-1].split(':')[1].strip()            # 但characteristics_ch1有的有两个值，有的有三个，取最后一个
            infection.append(infection_status)

            sample_gpl = gse.gsms[i].metadata['platform_id']
            gsm_list.append(sample_gpl)
            # 对数等的处理 # 可能不同做法
            if sample_gpl == ['GPL96']: #  MAS5-calculated Signal intensityzs
                value = pd.concat([value, sample_value.iloc[n-1]], axis=1)
                #部分样品跑偏严重
                # infection.pop() 
            elif sample_gpl == ['GPL570']: #  MAS5-calculated Signal intensity
                value = pd.concat([value, sample_value.iloc[n-1]], axis=1)
                # 直接代入导致分数分布大量超过1（最大值为1），所以先不用 infection.pop() 
            elif sample_gpl == ['GPL571']:
                value = pd.concat([value, 2**sample_value.iloc[n-1]], axis=1)
            elif sample_gpl == ['GPL2507']: #  Normalized value from BeadStudio software
                value = pd.concat([value, 2**sample_value.iloc[n-1]], axis=1)
                # 名不一样，大量nan
                # infection.pop() # 把刚刚加进去的感染状态也删了
            elif sample_gpl == ['GPL6947']:
                value = pd.concat([value, sample_value.iloc[n-1]], axis=1)
            # GPL6947 cubic spline normalised intensity
            elif sample_gpl == ['GPL10558']:
                value = pd.concat([value, 2**sample_value.iloc[n-1]], axis=1)
            elif sample_gpl == ['GPL6102']:
                value = pd.concat([value, sample_value.iloc[n-1]], axis=1)
            elif sample_gpl == ['GPL6244']:
                value = pd.concat([value, 2**sample_value.iloc[n-1]], axis=1)
            else:
                print(gse.gsms[i].columns["description"][1])
                print(sample_gpl)
                print('Now just use the raw value')
                print('Samples: ', sample_value.iloc[n-1].head())
                print('-'*50)
                value = pd.concat([value, sample_value.iloc[n-1]], axis=1)
                no_new_gpl = 0

        value = value.T

        # merge the infection status and value
        value.insert(0, 'infection_status', infection)
        value.insert(0, 'gsm', gsm_list)
        # Save the data and Show the first 5 rows
        value.to_csv('./datasets/' + GEONAME + '.csv', index=False)
        # value.head(1)
        # print first row and first 6 columns
        print(value.head(2).iloc[:, :6])
    except:
        print('Error in ' + GEONAME)
        print(gse.gsms[i].columns["description"][1])
        continue

GSE6269
['Age: 18m, when sample taken', 'Gender: F', 'Race: Caucasian', 'Illness:  Fever/URI', 'Treatment: Oseltamivir', 'Pathogen: Influenza A']
ID_REF           gsm infection_status  1007_s_at  1053_at  117_at  121_at
GSM173175   [GPL570]      Influenza A      306.2    727.1   998.5   808.3
GSM173176  [GPL2507]      Influenza A        NaN      NaN     NaN     NaN
GSE20346
['tissue: Paxgene Whole Blood RNA Samples']
ID_REF           gsm                 infection_status  ILMN_1651199  \
GSM509706  [GPL6947]  Paxgene Whole Blood RNA Samples     -11.70438   
GSM509707  [GPL6947]  Paxgene Whole Blood RNA Samples      -7.36877   

ID_REF     ILMN_1651209  ILMN_1651221  ILMN_1651228  
GSM509706      14.77135      12.49210      562.4998  
GSM509707      29.17356      10.29059      213.7863  
GSE40012
['tissue: PAXgene whole blood', 'sample type: systemic inflammatory response', 'day: 3', 'ID: 23', 'gender: male']
ID_REF           gsm infection_status  ILMN_1343291  ILMN_1343295  \
GSM983403 

In [ ]:
GEONAME="GSE42834"

In [ ]:
gse = GEOparse.get_GEO(geo=GEONAME, destdir="./datasets")
sample_value = gse.pivot_samples('VALUE').T

infection = []
gsm_list = []
n = 0

value = pd.DataFrame()
for i in gse.gsms:
    n += 1

    # 感染状态的处理
    characteristics_ch1 = gse.gsms[i].metadata['characteristics_ch1']   # the infection_status is in the characteristics_ch1
    infection_status = characteristics_ch1[-1].split(':')[1].strip()            # 但characteristics_ch1有的有两个值，有的有三个，取最后一个
    infection.append(infection_status)

    sample_gpl = gse.gsms[i].metadata['platform_id']
    gsm_list.append(sample_gpl)
    # 对数等的处理 # 可能不同做法
    if sample_gpl == ['GPL96']: #  MAS5-calculated Signal intensityzs
        value = pd.concat([value, sample_value.iloc[n-1]], axis=1)
        #部分样品跑偏严重
        # infection.pop() 
    elif sample_gpl == ['GPL570']: #  MAS5-calculated Signal intensity
        value = pd.concat([value, sample_value.iloc[n-1]], axis=1)
        # 直接代入导致分数分布大量超过1（最大值为1），所以先不用 infection.pop() 
    elif sample_gpl == ['GPL571']:
        value = pd.concat([value, 2**sample_value.iloc[n-1]], axis=1)
    elif sample_gpl == ['GPL2507']: #  Normalized value from BeadStudio software
        value = pd.concat([value, 2**sample_value.iloc[n-1]], axis=1)
        # 名不一样，大量nan
        # infection.pop() # 把刚刚加进去的感染状态也删了
    elif sample_gpl == ['GPL6947']:
        value = pd.concat([value, sample_value.iloc[n-1]], axis=1)
    # GPL6947 cubic spline normalised intensity
    else:
        print(gse.gsms[i].columns["description"][1])
        print(sample_gpl)
        print('Now just use the raw value')
        print('Samples: ', sample_value.iloc[n-1].head())
        print('-'*50)
        value = pd.concat([value, sample_value.iloc[n-1]], axis=1)

value = value.T

# merge the infection status and value
value.insert(0, 'infection_status', infection)
value.insert(0, 'gsm', gsm_list)
print(value.head())
# Save the data and Show the first 5 rows
value.to_csv('./datasets/' + GEONAME + '.csv', index=False)
value.head()

OSError: Download failed due to 'Downloaded size do not match the expected size for ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE42nnn/GSE42834/soft/GSE42834_family.soft.gz'. ID could be incorrect or the data might not be public yet.

In [ ]:
# Save the data and Show the first 5 rows
value.to_csv('./datasets/' + GEONAME + '.csv', index=False)
value.head()

## 不用了

不清楚什么时候再把这个id转化成通用的Gene Symbol。

可能在这里就转，也可以分析结束了再转。

大概需要10分钟

In [ ]:
# get all Series_platform_id
gpls = gse.metadata['platform_id']
    

In [ ]:
# GPL571 = gse.gpls['GPL571']
# n = 0
# for rows in sample_value:
#     Gene_Symbol = GPL571.table[GPL571.table['ID'] == rows]['Gene Symbol']
#     if Gene_Symbol.empty:
#         Gene_Symbol[0] = rows
#     sample_value.rename(columns={rows: Gene_Symbol.values[0]}, inplace=True)
#     n += 1
#     if n % 100 == 0:
#         print("Finished %d rows of %d at %s" % (n, sample_value.shape[1], datetime.datetime.now()))
# sample_value.tail()

In [ ]:
# gse.gpls['GPL571'].table.columns # GPL571, 也就是这个GEO数据集的平台，的列。
# # 从这个平台可以直接读取Gene Symbol，Target Description等等有用的东西。

In [ ]:
# gse.gpls['GPL571'].table.head() # 前5行